# 1. Import libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_auc_score, confusion_matrix
from IPython.display import clear_output

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import tensorflow as tf

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

# 2. Import datasets

In [4]:
df = pd.read_csv("../inputs/1_Bins.csv")
df0 = df[df['Def']==0].reset_index(drop=True)
df1 = df[df['Def']==1].reset_index(drop=True)

df01 = df0[:][:3000]
df02 = df0[:][3000:6000]
df03 = df0[:][6000:]

df11 = df1[:][:500]
df12 = df1[:][500:1000]
df13 = df1[:][1000:]

df1 = pd.concat([df01,df11])
df2 = pd.concat([df02,df12])
df3 = pd.concat([df03,df13])

del df0, df01, df02, df03, df11, df12, df13
df.shape

(9578, 13)

# 3. Model - 1st

In [5]:
np.random.seed(108)
activation_c = []
optimizer_c = []
l2_c = []
dropout_c = []
epochs_c = []
batch_size_c = []
auc_c = []

In [6]:
print('activation\toptimizer\tl2\tdropout\tepochs\tbatch_size\tauc')
for a in ['tanh']:
    for o in ['adam']:
        for l in [0.0001,0.001, 0.01]:
            for d in [0.0001,0.001, 0.01]:
                for e in [500, 1000, 5000]:
                    for b in [50, 100, 500]:
                        model = Sequential() 
                        model.add(Dense(10, activation=a, input_dim=10, kernel_regularizer=l2(l))) 
                        model.add(Dropout(d))
                        model.add(Dense(10, activation=a, kernel_regularizer=l2(l)))
                        model.add(Dropout(d))
                        model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l)))
                        model.compile(optimizer=o, loss='binary_crossentropy', metrics='AUC') 
                        
                        h = []
                        for i in [0,1,2]:
                            if i==0: train=pd.concat([df1,df2])
                            elif i==2: train=pd.concat([df2,df3])
                            else: train=pd.concat([df3,df1])
                            y = train['Def']
                            x = train.drop(['delinq_2yrs_bin','int_rate_bin','Def'], axis=1)
                            model.fit(x,y,epochs=e,batch_size=b,verbose=0,callbacks=EarlyStopping(monitor='loss',patience=3))
                            
                            if i==0: test=df3
                            elif i==2: test=df1
                            else: test=df2
                            y = test['Def']
                            x = test.drop(['delinq_2yrs_bin','int_rate_bin','Def'], axis=1)
                            pred_values = model.predict(x,batch_size=b,verbose=0)
                            
                            pred_values = pd.DataFrame(pred_values)[0]
                            h.append(np.round(roc_auc_score(y, pred_values), 3))
                        clear_output(wait=False)
                        print('activation\toptimizer\tl2\tdropout\tepochs\tbatch_size\tauc')
                        print(a,'\t\t',o,'\t\t',l,'\t',d,'\t',e,'\t',b,'\t\t',np.round(np.mean(h), 3))
                        
                        activation_c.append(a)
                        optimizer_c.append(o)
                        l2_c.append(l)
                        dropout_c.append(d)
                        epochs_c.append(e)
                        batch_size_c.append(b)
                        auc_c.append(np.round(np.mean(h), 3))

activation	optimizer	l2	dropout	epochs	batch_size	auc
tanh 		 adam 		 0.01 	 0.01 	 5000 	 500 		 0.498


In [7]:
train = pd.DataFrame({'activation':activation_c, 'optimizer':optimizer_c, 'l2':l2_c, 'dropout':dropout_c,
                      'epochs':epochs_c, 'batch_size':batch_size_c, 'auc':auc_c})
test = train[train['auc']==train['auc'].max()]
test

,activation,optimizer,l2,dropout,epochs,batch_size,auc
27,tanh,adam,0.001,0.0001,500,50,0.659
30,tanh,adam,0.001,0.0001,1000,50,0.659


# 4. Model - 2nd

In [8]:
np.random.seed(108)
activation_c = []
optimizer_c = []
l2_c = []
dropout_c = []
epochs_c = []
batch_size_c = []
auc_c = []

In [9]:
for a in ['tanh']:
    for o in ['adam']:
        for l in [0.001]:
            for d in [0.0001,0.0005,0.001]:
                for e in [500, 1000, 5000]:
                    for b in [50, 100, 500]:
                        model = Sequential() 
                        model.add(Dense(10, activation=a, input_dim=10, kernel_regularizer=l2(l))) 
                        model.add(Dropout(d))
                        model.add(Dense(10, activation=a, kernel_regularizer=l2(l)))
                        model.add(Dropout(d))
                        model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l)))
                        model.compile(optimizer=o, loss='binary_crossentropy', metrics='AUC') 
                        
                        h = []
                        for i in [0,1,2]:
                            if i==0: train=pd.concat([df1,df2])
                            elif i==2: train=pd.concat([df2,df3])
                            else: train=pd.concat([df3,df1])
                            y = train['Def']
                            x = train.drop(['delinq_2yrs_bin','int_rate_bin','Def'], axis=1)
                            model.fit(x,y,epochs=e,batch_size=b,verbose=0,callbacks=EarlyStopping(monitor='loss',patience=3))
                            
                            if i==0: test=df3
                            elif i==2: test=df1
                            else: test=df2
                            y = test['Def']
                            x = test.drop(['delinq_2yrs_bin','int_rate_bin','Def'], axis=1)
                            pred_values = model.predict(x,batch_size=b,verbose=0)
                            
                            pred_values = pd.DataFrame(pred_values)[0]
                            h.append(np.round(roc_auc_score(y, pred_values), 3))
                        clear_output(wait=False)
                        print('activation\toptimizer\tl2\tdropout\tepochs\tbatch_size\tauc')
                        print(a,'\t\t',o,'\t\t',l,'\t',d,'\t',e,'\t',b,'\t\t',np.round(np.mean(h), 3))
                        
                        activation_c.append(a)
                        optimizer_c.append(o)
                        l2_c.append(l)
                        dropout_c.append(d)
                        epochs_c.append(e)
                        batch_size_c.append(b)
                        auc_c.append(np.round(np.mean(h), 3))

activation	optimizer	l2	dropout	epochs	batch_size	auc
tanh 		 adam 		 0.001 	 0.001 	 5000 	 500 		 0.634


In [10]:
train = pd.DataFrame({'activation':activation_c, 'optimizer':optimizer_c, 'l2':l2_c, 'dropout':dropout_c,
                      'epochs':epochs_c, 'batch_size':batch_size_c, 'auc':auc_c})
test = train[train['auc']==train['auc'].max()]
test

,activation,optimizer,l2,dropout,epochs,batch_size,auc
0,tanh,adam,0.001,0.0001,500,50,0.659
12,tanh,adam,0.001,0.0005,1000,50,0.659


# 5. Model - 3rd

In [11]:
np.random.seed(108)
activation_c = []
optimizer_c = []
l2_c = []
dropout_c = []
epochs_c = []
batch_size_c = []
auc_c = []

In [12]:
for a in ['tanh']:
    for o in ['adam']:
        for l in [0.001]:
            for d in [0.0005]:
                for e in [100,200,500,700,1000]:
                    for b in [50,100,200]:
                        model = Sequential() 
                        model.add(Dense(10, activation=a, input_dim=10, kernel_regularizer=l2(l))) 
                        model.add(Dropout(d))
                        model.add(Dense(10, activation=a, kernel_regularizer=l2(l)))
                        model.add(Dropout(d))
                        model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l)))
                        model.compile(optimizer=o, loss='binary_crossentropy', metrics='AUC') 
                        
                        h = []
                        for i in [0,1,2]:
                            if i==0: train=pd.concat([df1,df2])
                            elif i==2: train=pd.concat([df2,df3])
                            else: train=pd.concat([df3,df1])
                            y = train['Def']
                            x = train.drop(['delinq_2yrs_bin','int_rate_bin','Def'], axis=1)
                            model.fit(x,y,epochs=e,batch_size=b,verbose=0,callbacks=EarlyStopping(monitor='loss',patience=3))
                            
                            if i==0: test=df3
                            elif i==2: test=df1
                            else: test=df2
                            y = test['Def']
                            x = test.drop(['delinq_2yrs_bin','int_rate_bin','Def'], axis=1)
                            pred_values = model.predict(x,batch_size=b,verbose=0)
                            
                            pred_values = pd.DataFrame(pred_values)[0]
                            h.append(np.round(roc_auc_score(y, pred_values), 3))
                        clear_output(wait=False)
                        print('activation\toptimizer\tl2\tdropout\tepochs\tbatch_size\tauc')
                        print(a,'\t\t',o,'\t\t',l,'\t',d,'\t',e,'\t',b,'\t\t',np.round(np.mean(h), 3))
                        
                        activation_c.append(a)
                        optimizer_c.append(o)
                        l2_c.append(l)
                        dropout_c.append(d)
                        epochs_c.append(e)
                        batch_size_c.append(b)
                        auc_c.append(np.round(np.mean(h), 3))

activation	optimizer	l2	dropout	epochs	batch_size	auc
tanh 		 adam 		 0.001 	 0.0005 	 1000 	 200 		 0.655


In [13]:
train = pd.DataFrame({'activation':activation_c, 'optimizer':optimizer_c, 'l2':l2_c, 'dropout':dropout_c,
                      'epochs':epochs_c, 'batch_size':batch_size_c, 'auc':auc_c})
test = train[train['auc']==train['auc'].max()]
test

,activation,optimizer,l2,dropout,epochs,batch_size,auc
7,tanh,adam,0.001,0.0005,500,100,0.657
